## 1. <a id='toc1_'></a>[Imports](#toc0_)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import numpy as np
import torch
import pickle
import time
import matplotlib.pyplot as plt
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
plt.rcParams.update({"axes.grid" : True, "grid.color": "black", "grid.alpha":"0.25", "grid.linestyle": "--"})
plt.rcParams.update({'font.size': 14})

In [3]:
from BufferStockModel import BufferStockModelClass

from BufferStockModelEGM import BufferStockModelEGMClass
from EconDLSolvers import choose_gpu

## 2. <a id='toc1_'></a>[Model](#toc0_)

In [4]:
device = choose_gpu()

No GPU available, using CPU


In [36]:
algoname = "DeepSimulate"
model = BufferStockModelClass(device=device,algoname=algoname)

In [21]:
algoname = "SimEGM"
model_SimEGM = BufferStockModelClass(device=device,algoname=algoname)

In [37]:
model.simulate_R()
print(model.sim.R.item())

-1.7687537670135498


## 3. <a id='toc1_'></a>[Solve](#toc0_)

In [ ]:
model.train.K_time = 60
model.solve(do_print=True)
model.compute_euler_errors(Nbatch_share=0.001)
model.save('GUV_DeepSimulate_base.pkl')

started solving: 2025-05-05 14:11:37


In [ ]:
model_SimEGM.train.K_time = 60
model_SimEGM.solve(do_print=True)
model_SimEGM.compute_euler_errors(Nbatch_share=0.001)
model_SimEGM.save('GUV_SimEGM_base.pkl')


### Approximate model

In [ ]:
par_dict = {'use_reg':True}
algoname = "DeepSimulate"
model_reg = BufferStockModelClass(device=device,algoname=algoname,par=par_dict)

In [ ]:
par_dict = {'use_reg':True}
algoname = "SimEGM"
model_SimEGM_reg = BufferStockModelClass(device=device,algoname=algoname,par=par_dict)

In [ ]:
model_reg.train.K_time = 60
model_reg.solve(do_print=True)
model_reg.compute_euler_errors(Nbatch_share=0.001)
model_reg.save('GUV_DeepSimulate_reg.pkl')

In [ ]:
model_SimEGM_reg.train.K_time = 60
model_SimEGM_reg.solve(do_print=True)
model_SimEGM_reg.compute_euler_errors(Nbatch_share=0.001)
model_SimEGM_reg.save('GUV_SimEGM_reg.pkl')

### Policy Results

In [ ]:
model_EGM = BufferStockModelEGMClass()

In [ ]:
sol_con_grid,sol_sr_grid = model_reg.compute_policy_on_grids(model_EGM.egm.m_grids, model_EGM.egm.z_grid)

In [ ]:
sol_con_grid_SimEGM,sol_sr_grid = model_SimEGM_reg.compute_policy_on_grids(model_EGM.egm.m_grids, model_EGM.egm.z_grid)

In [ ]:
# Organize results into a dictionary
results_dict = {
    'sol_con_grid_DeepSimulate': sol_con_grid,
    'sol_con_grid_SimEGM': sol_con_grid_SimEGM,
}

# Save to pickle
with open('policy_results.pkl', 'wb') as f:
    pickle.dump(results_dict, f)